Imports & Dependencies: Load all necessary libraries (pandas, ta, binance, etc.) for trading and data analysis.

In [ ]:
import os
import time
from datetime import datetime, timedelta
from Key import api, secret
from binance.um_futures import UMFutures
import ta
import pandas as pd
from time import sleep
from binance.error import ClientError, ParameterRequiredError
import traceback
from dotenv import load_dotenv

Configuration: Define trading parameters like take-profit levels, stop-loss, and bot instance IDs.

In [ ]:
load_dotenv()

api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_API_SECRET')
client = UMFutures(key=api_key, secret=api_secret)

BOT_INSTANCE_ID = 'BTCSLOWFASTEMA'
BOT_NAME = os.getenv('BOT_NAME', f'Bot_{BOT_INSTANCE_ID}')

MAX_TRADES_PER_DAY = 3
TRADE_COUNTER_FILE = f"{BOT_NAME}_daily_trades.json"

# Trading parameters
tp_levels = {
    'tp1': {'percentage': 0.02, 'quantity_pct': 0.40},  # 2% → Close 40%
    'tp2': {'percentage': 0.02, 'quantity_pct': 0.40},  # 4% → Close 40%
    'tp3': {'percentage': 0.035, 'quantity_pct': 0.20},  # 6.1% → Close 20%
}

sl_normal = 0.01  # Normal stop loss percentage
balance_percentage = 0.13  # Base percentage of balance to use
leverage = 1
margin_type = 'CROSS'
qty_limit = 1  # Maximum number of simultaneous positions
candle_interval_integreted = 240
candle_interval = '4h'

# Volatility spike detection parameters
vol_threshold = 3  # Multiplier for detecting spikes
pause_duration = 24  # Bars to pause after spike (24 bars = 6 hours on 15m)
vol_lookback = 10  # Period for calculating volatility (48 bars = 12 hours)
use_returns_volatility = True

# Bot state tracking
symbol_states = {}
tracked_positions = {}
volatility_states = {}

# ET market close time (16:00 ET)
ET_MARKET_CLOSE_HOUR = 16
trailing_stops = {}
symbol_states = {}
bot_positions = {}

API Connectivity: A robust wrapper for Binance API calls that handles retries, rate limits, and network errors.

In [ ]:
def api_call_with_retry(func, max_retries=3, delay=2, **kwargs):
    """
    Wrapper for API calls with retry logic
    Enhanced for better order cancellation handling
    """
    func_name = getattr(func, '__name__', str(func))

    for attempt in range(max_retries):
        try:
            print(f"🔄 API Call: {func_name} (attempt {attempt + 1}/{max_retries})")
            if kwargs:
                print(f"📊 Parameters: {kwargs}")

            result = func(**kwargs)

            # Log successful calls
            print(f"✅ {func_name} successful")
            if result and isinstance(result, (list, dict)):
                if isinstance(result, list):
                    print(f"📊 Returned {len(result)} items")
                else:
                    print(f"📊 Returned: {type(result).__name__}")

            return result

        except ClientError as error:
            error_code = getattr(error, 'error_code', 'Unknown')
            error_message = getattr(error, 'error_message', str(error))

            print(f"❌ Binance API Error in {func_name} (attempt {attempt + 1}/{max_retries}):")
            print(f"   Code: {error_code}")
            print(f"   Message: {error_message}")

            # Handle specific error codes that shouldn't be retried
            if error_code in [-2011, -1013, -4045]:  # Order not found, invalid quantity, etc.
                print(f"🚫 Non-retryable error code {error_code}, stopping retries")
                # For order cancellation, we might want to know if order doesn't exist
                if 'cancel' in func_name.lower() and error_code == -2011:
                    print(f"ℹ️ Order already cancelled or doesn't exist")
                    return {'msg': 'Order not found', 'cancelled': False}
                return None

            # Rate limiting - longer delay
            if error_code in [-1003, -1015]:  # Too many requests, too frequent
                print(f"⏱️ Rate limiting detected, using longer delay")
                if attempt < max_retries - 1:
                    sleep_time = delay * (attempt + 2) * 2  # Longer delay for rate limits
                    print(f"😴 Sleeping {sleep_time}s due to rate limiting...")
                    sleep(sleep_time)
                continue

            if attempt < max_retries - 1:
                sleep_time = delay * (attempt + 1)
                print(f"😴 Retrying in {sleep_time}s...")
                sleep(sleep_time)
            else:
                print(f"❌ Max retries reached for {func_name}")
                return None

        except ParameterRequiredError as e:
            print(f"❌ Parameter Error in {func_name}: {str(e)}")
            print(f"🔍 Check if you're using the correct API method for your use case")
            # Don't retry parameter errors - they won't succeed
            return None

        except ConnectionError as e:
            print(f"🌐 Connection Error in {func_name} (attempt {attempt + 1}/{max_retries}): {str(e)}")
            if attempt < max_retries - 1:
                sleep_time = delay * (attempt + 1)
                print(f"😴 Retrying connection in {sleep_time}s...")
                sleep(sleep_time)
            else:
                print(f"❌ Connection failed after {max_retries} attempts")
                return None

        except Exception as e:
            print(f"⚠️ Unexpected error in {func_name} (attempt {attempt + 1}/{max_retries}): {str(e)}")
            print(f"🔍 Error type: {type(e).__name__}")

            # Print traceback for debugging
            import traceback
            traceback.print_exc()

            if attempt < max_retries - 1:
                sleep_time = delay
                print(f"😴 Retrying in {sleep_time}s...")
                sleep(sleep_time)
            else:
                print(f"❌ Max retries reached for {func_name} due to unexpected error")
                return None

    return None

Market Data & Account Utilities: Functions to fetch wallet balances, candlestick data (OHLCV), and asset-specific precision.

In [ ]:
def get_balance_usdt():
    """Get USDT futures balance"""
    try:
        response = api_call_with_retry(client.balance, recvWindow=6000)
        if response:
            for elem in response:
                if elem['asset'] == 'BNFCR':
                    balance = float(elem['balance'])
                    base_volume = balance * balance_percentage
                    return balance, base_volume
    except Exception as e:
        print(f"Error getting balance: {e}")
    return None, None


def klines(symbol, interval='15m', limit=500):
    """Get candlestick data"""
    try:
        resp = pd.DataFrame(client.klines(symbol, interval, limit=limit))
        resp = resp.iloc[:, :6]
        resp.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
        resp = resp.set_index('Time')
        resp.index = pd.to_datetime(resp.index, unit='ms')
        resp = resp.astype(float)
        return resp
    except Exception as error:
        print(f"Error getting klines for {symbol}: {error}")
        return None


def get_price_precision(symbol):
    """Get price precision for symbol"""
    try:
        resp = client.exchange_info()['symbols']
        for elem in resp:
            if elem['symbol'] == symbol:
                return elem['pricePrecision']
    except Exception as e:
        print(f"Error getting price precision: {e}")
    return 4


def get_qty_precision(symbol):
    """Get quantity precision for symbol"""
    try:
        resp = client.exchange_info()['symbols']
        for elem in resp:
            if elem['symbol'] == symbol:
                return elem['quantityPrecision']
    except Exception as e:
        print(f"Error getting qty precision: {e}")
    return 3

Order Preparation: Utilities to configure the exchange (Hedge mode, leverage) and format numbers to meet API requirements.

In [ ]:
def set_leverage(symbol, level):
    """Set leverage for symbol"""
    try:
        response = api_call_with_retry(client.change_leverage,
                                       symbol=symbol, leverage=level, recvWindow=6000)
        if response:
            print(f"Leverage set to {level}x for {symbol}")
    except Exception as error:
        print(f"Error setting leverage: {error}")


def set_mode(symbol, margin_type):
    """Set margin type for symbol"""
    try:
        response = api_call_with_retry(client.change_margin_type,
                                       symbol=symbol, marginType=margin_type, recvWindow=6000)
        if response:
            print(f"Margin type set to {margin_type} for {symbol}")
    except Exception as error:
        print(f"Error setting margin type: {error}")


def validate_and_format_order_params(symbol, price=None, quantity=None, stop_price=None):
    """Validate and format order parameters"""
    try:
        price_precision = get_price_precision(symbol)
        qty_precision = get_qty_precision(symbol)

        result = {}

        if price is not None:
            result['price'] = f"{price:.{price_precision}f}"
        if quantity is not None:
            result['quantity'] = f"{quantity:.{qty_precision}f}"
        if stop_price is not None:
            result['stop_price'] = f"{stop_price:.{price_precision}f}"

        return result
    except Exception as e:
        print(f"Error validating order params: {e}")
        return None

Trailing Stop Logic: The core logic for tracking price movement and "trailing" the stop-loss order to lock in profits.

In [ ]:
def update_trailing_stops():
    """
    Update all active trailing stops based on current market prices
    Compatible with hedge mode and bot instance tracking
    """
    try:
        if not trailing_stops:
            return

        print(f"\n🔄 Checking {len(trailing_stops)} trailing stops...")

        for position_key, stop_data in list(trailing_stops.items()):
            symbol = stop_data['symbol']
            side = stop_data['side']
            position_side = stop_data['position_side']
            entry_price = stop_data['entry_price']
            current_sl_price = stop_data['current_sl_price']
            current_sl_order_id = stop_data.get('current_sl_order_id')
            trailing_distance = stop_data.get('trailing_distance', sl_normal)

            # Get current price
            ticker_response = api_call_with_retry(client.ticker_price, symbol=symbol)
            if not ticker_response:
                print(f"⚠️ Could not get current price for {symbol}")
                continue

            current_price = float(ticker_response['price'])

            # Calculate new stop loss price based on trailing logic
            new_sl_price = None
            should_update = False

            if side == 'buy':  # LONG position
                # Track highest price since entry
                if 'highest_price' not in stop_data:
                    stop_data['highest_price'] = entry_price

                if current_price > stop_data['highest_price']:
                    stop_data['highest_price'] = current_price

                # Calculate new trailing stop (distance below highest price)
                potential_sl = stop_data['highest_price'] * (1 - trailing_distance)

                # Only update if new SL is higher than current SL (trailing up)
                if potential_sl > current_sl_price:
                    new_sl_price = potential_sl
                    should_update = True

            else:  # SHORT position (side == 'sell')
                # Track lowest price since entry
                if 'lowest_price' not in stop_data:
                    stop_data['lowest_price'] = entry_price

                if current_price < stop_data['lowest_price']:
                    stop_data['lowest_price'] = current_price

                # Calculate new trailing stop (distance above lowest price)
                potential_sl = stop_data['lowest_price'] * (1 + trailing_distance)

                # Only update if new SL is lower than current SL (trailing down)
                if potential_sl < current_sl_price:
                    new_sl_price = potential_sl
                    should_update = True

            if should_update and new_sl_price:
                print(f"🎯 Updating trailing stop for {symbol} ({position_side})")
                print(f"   Current price: {current_price:.6f}")
                print(f"   Old SL: {current_sl_price:.6f}")
                print(f"   New SL: {new_sl_price:.6f}")

                # Cancel existing stop loss order
                if current_sl_order_id:
                    cancel_params = {
                        'symbol': symbol,
                        'orderId': current_sl_order_id,
                        'recvWindow': 6000
                    }
                    cancel_response = api_call_with_retry(client.cancel_order, **cancel_params)
                    if cancel_response:
                        print(f"✅ Cancelled old SL order {current_sl_order_id}")
                    else:
                        print(f"⚠️ Could not cancel old SL order {current_sl_order_id}")

                    # Small delay to ensure cancellation processes
                    time.sleep(1)

                # Place new trailing stop order
                sl_order_side = 'SELL' if side == 'buy' else 'BUY'

                # Validate new stop price and quantity
                validation = validate_and_format_order_params(
                    symbol,
                    stop_price=new_sl_price,
                    quantity=stop_data['quantity']
                )

                if validation:
                    new_sl_params = {
                        'symbol': symbol,
                        'side': sl_order_side,
                        'positionSide': position_side,
                        'type': 'STOP_MARKET',
                        'quantity': validation['quantity'],
                        'stopPrice': validation['stop_price'],
                        'timeInForce': 'GTC',
                        'recvWindow': 10000,
                        'newClientOrderId': f"{BOT_INSTANCE_ID}_TSL_{position_side}_{int(time.time())}"
                    }

                    new_sl_response = api_call_with_retry(client.new_order, **new_sl_params)

                    if new_sl_response:
                        # Update trailing stop data
                        stop_data['current_sl_price'] = new_sl_price
                        stop_data['current_sl_order_id'] = new_sl_response['orderId']
                        print(f"✅ New trailing stop placed: {new_sl_price:.6f}")
                    else:
                        print(f"❌ Failed to place new trailing stop")
                else:
                    print(f"❌ Failed to validate new stop loss parameters")

    except Exception as e:
        print(f"❌ Error in update_trailing_stops: {str(e)}")
        import traceback
        traceback.print_exc()

def handle_trailing_stops(symbol, current_price, position_amt):
    """
    Handle trailing stops for bot positions
    """
    try:
        # Find the correct position side based on position amount
        position_side = 'LONG' if position_amt > 0 else 'SHORT'
        bot_position_key = f"{symbol}_{BOT_INSTANCE_ID}_{position_side}"

        if bot_position_key not in trailing_stops:
            return  # No trailing stop for this position

        ts_data = trailing_stops[bot_position_key]

        # Skip if no current stop loss order
        if not ts_data.get('current_sl_order_id'):
            return

        # Get current market price
        ticker = api_call_with_retry(client.ticker_price, symbol=symbol)
        if not ticker:
            return

        current_price = float(ticker['price'])
        entry_price = ts_data['entry_price']
        side = ts_data['side']
        trailing_distance = ts_data.get('trailing_distance', sl_normal)  # Fallback to normal SL

        should_update = False
        new_sl_price = None

        print(f"🔍 Checking trailing stop for {symbol} {position_side}:")
        print(f"   Current price: {current_price:.6f}")
        print(f"   Entry price: {entry_price:.6f}")
        print(f"   Current SL: {ts_data['current_sl_price']:.6f}")

        if side == 'buy':  # LONG position
            # Update highest price seen
            if ts_data['highest_price'] is None or current_price > ts_data['highest_price']:
                ts_data['highest_price'] = current_price

            # Calculate new stop loss (trail below highest price)
            new_sl_price = ts_data['highest_price'] * (1 - trailing_distance)

            # Only update if new SL is higher than current SL (tighter stop)
            if new_sl_price > ts_data['current_sl_price']:
                should_update = True
                print(f"   New high: {ts_data['highest_price']:.6f} -> SL: {new_sl_price:.6f}")

        else:  # SHORT position
            # Update lowest price seen
            if ts_data['lowest_price'] is None or current_price < ts_data['lowest_price']:
                ts_data['lowest_price'] = current_price

            # Calculate new stop loss (trail above lowest price)
            new_sl_price = ts_data['lowest_price'] * (1 + trailing_distance)

            # Only update if new SL is lower than current SL (tighter stop)
            if new_sl_price < ts_data['current_sl_price']:
                should_update = True
                print(f"   New low: {ts_data['lowest_price']:.6f} -> SL: {new_sl_price:.6f}")

        if should_update and new_sl_price:
            # Cancel current stop loss order
            try:
                cancel_response = api_call_with_retry(
                    client.cancel_order,
                    symbol=symbol,
                    orderId=ts_data['current_sl_order_id'],
                    recvWindow=10000
                )
                if cancel_response:
                    print(f"✅ Cancelled old SL order: {ts_data['current_sl_order_id']}")
                else:
                    print(f"⚠️ Could not cancel old SL order: {ts_data['current_sl_order_id']}")
            except Exception as e:
                print(f"⚠️ Error cancelling old SL: {e}")

            # Validate new stop loss price
            sl_price_validation = validate_and_format_order_params(
                symbol,
                stop_price=new_sl_price,
                quantity=abs(position_amt)
            )

            if sl_price_validation:
                # Determine order parameters
                sl_order_side = 'SELL' if side == 'buy' else 'BUY'

                # Place new trailing stop loss order
                sl_params = {
                    'symbol': symbol,
                    'side': sl_order_side,
                    'positionSide': position_side,
                    'type': 'STOP_MARKET',
                    'quantity': sl_price_validation['quantity'],
                    'stopPrice': sl_price_validation['stop_price'],
                    'timeInForce': 'GTC',
                    'recvWindow': 10000,
                    'newClientOrderId': f"{BOT_INSTANCE_ID}_TSL_{position_side}_{int(time.time())}"
                }

                sl_response = api_call_with_retry(client.new_order, **sl_params)

                if sl_response:
                    # Update trailing stop data
                    ts_data['current_sl_price'] = new_sl_price
                    ts_data['current_sl_order_id'] = sl_response['orderId']

                    direction = '-' if side == 'buy' else '+'
                    improvement = abs((new_sl_price - ts_data['current_sl_price']) / entry_price * 100)

                    print(f"🎯 Trailing stop updated for {symbol} {position_side}:")
                    print(f"   New SL: {new_sl_price:.6f} ({direction}{trailing_distance * 100:.2f}%)")
                    print(f"   Order ID: {sl_response['orderId']}")

                    # Save updated trailing stop data
                    trailing_stops[bot_position_key] = ts_data

                else:
                    print(f"❌ Failed to place new trailing stop loss for {symbol} {position_side}")
            else:
                print(f"❌ Invalid stop loss parameters for {symbol}")

    except Exception as e:
        print(f"❌ Error in handle_trailing_stops for {symbol}: {str(e)}")
        import traceback
        traceback.print_exc()

def update_trailing_stop_monitoring():
    """
    Main function to check and update all trailing stops
    Call this in your main loop
    """
    try:
        if not trailing_stops:
            return

        print(f"🔄 Checking {len(trailing_stops)} trailing stops...")

        # Get all current positions
        positions = api_call_with_retry(client.get_position_risk, recvWindow=10000)
        if not positions:
            return

        # Check each tracked trailing stop
        for bot_position_key, ts_data in list(trailing_stops.items()):
            symbol = ts_data['symbol']
            position_side = ts_data['position_side']

            # Find corresponding position
            current_position = None
            for pos in positions:
                if (pos['symbol'] == symbol and
                        pos['positionSide'] == position_side):
                    position_amt = float(pos['positionAmt'])
                    if abs(position_amt) > 0:  # Position still exists
                        current_position = pos
                        break

            if current_position:
                # Position still exists, update trailing stop
                position_amt = float(current_position['positionAmt'])
                current_price = float(current_position['markPrice'])

                handle_trailing_stops(symbol, current_price, position_amt)
            else:
                # Position closed, remove from trailing stops
                print(f"📝 Position closed, removing trailing stop: {bot_position_key}")
                del trailing_stops[bot_position_key]

    except Exception as e:
        print(f"❌ Error in update_trailing_stop_monitoring: {e}")
        import traceback
        traceback.print_exc()

Order Execution: Functions to enter trades, manage reversals by closing opposite positions, and clean up open orders.

In [ ]:
def open_order(symbol, side, balance):
    """
    Open order without TP/SL - simplified for MA crossover strategy
    """
    try:
        # Set trading mode to hedge
        set_mode(symbol, 'HEDGE')
        set_leverage(symbol, leverage)

        # Calculate position size
        volume = balance * balance_percentage * leverage

        # Get current price
        ticker_response = api_call_with_retry(client.ticker_price, symbol=symbol)
        if not ticker_response:
            print(f"Could not get price for {symbol}")
            return None

        price = float(ticker_response['price'])
        qty = volume / price

        # Validate quantity
        qty_validation = validate_and_format_order_params(symbol, quantity=qty)
        if not qty_validation:
            print(f"Failed to validate quantity for {symbol}")
            return None

        qty = float(qty_validation['quantity'])

        print(f"\n🚀 Opening {side.upper()} position for {symbol}")
        print(f"   Position size: {qty}")
        print(f"   Current price: {price:.6f}")

        # HEDGE MODE: Set position side and order side
        position_side = 'LONG' if side == 'buy' else 'SHORT'
        order_side = 'BUY' if side == 'buy' else 'SELL'

        # Place market entry order
        order_params = {
            'symbol': symbol,
            'side': order_side,
            'positionSide': position_side,
            'type': 'MARKET',
            'quantity': qty_validation['quantity'],
            'recvWindow': 10000,
            'newClientOrderId': f"{BOT_INSTANCE_ID}_ENTRY_{position_side}_{int(time.time())}"
        }

        print(f"📝 Placing market entry order ({position_side})...")
        entry_response = api_call_with_retry(client.new_order, **order_params)

        if not entry_response:
            print("❌ Failed to place entry order")
            return None

        print("✅ Entry order placed successfully")

        # Get actual entry price
        sleep(2)
        positions = api_call_with_retry(client.get_position_risk, recvWindow=10000)

        actual_entry_price = None
        if positions:
            for position in positions:
                if (position['symbol'] == symbol and
                        position['positionSide'] == position_side):
                    pos_amt = float(position['positionAmt'])
                    if abs(pos_amt) > 0:
                        actual_entry_price = float(position['entryPrice'])
                        break

        if not actual_entry_price:
            actual_entry_price = price

        print(f"✅ Entry confirmed at: {actual_entry_price:.6f} ({position_side})")

        # Track position
        track_position(symbol, qty, actual_entry_price, side, position_side)

        # Update symbol state
        if symbol not in symbol_states:
            symbol_states[symbol] = {}

        symbol_states[symbol][position_side] = {
            'entry_time': datetime.now(),
            'entry_price': actual_entry_price,
            'position_side': position_side,
            'side': side,
            'quantity': qty,
            'last_signal_time': symbol_states.get(symbol, {}).get(position_side, {}).get('last_signal_time')
        }

        print(f"\n📊 {symbol} Position Summary ({position_side}):")
        print(f"   Direction: {side.upper()}")
        print(f"   Size: {qty}")
        print(f"   Entry: {actual_entry_price:.6f}")
        print(f"   Strategy: MA Crossover + ADX")

        return entry_response

    except Exception as e:
        print(f"❌ Error in open_order: {str(e)}")
        traceback.print_exc()
        return None


def close_opposite_position(symbol, new_signal_side):
    """
    Close opposite position when strategy signals a reversal

    Args:
        symbol: Trading symbol
        new_signal_side: 'buy' or 'sell' - the new signal direction

    Returns:
        bool: True if opposite position was closed, False otherwise
    """
    try:
        # Determine which position side to close
        position_side_to_close = 'SHORT' if new_signal_side == 'buy' else 'LONG'

        print(f"\n🔄 Checking for opposite position to close...")
        print(f"   New signal: {new_signal_side.upper()}")
        print(f"   Looking to close: {position_side_to_close}")

        # Get current positions
        positions = api_call_with_retry(client.get_position_risk, recvWindow=10000)
        if not positions:
            print("   No positions found")
            return False

        # Find opposite position
        for position in positions:
            if position['symbol'] == symbol and position['positionSide'] == position_side_to_close:
                position_amt = float(position['positionAmt'])

                if abs(position_amt) > 0:
                    print(f"   ✅ Found {position_side_to_close} position: {position_amt}")

                    # Cancel all orders for this position
                    cancel_orders_for_position(symbol, position_side_to_close)

                    # Close the position with market order
                    close_side = 'BUY' if position_side_to_close == 'SHORT' else 'SELL'

                    qty_validation = validate_and_format_order_params(
                        symbol,
                        quantity=abs(position_amt)
                    )

                    if qty_validation:
                        close_params = {
                            'symbol': symbol,
                            'side': close_side,
                            'positionSide': position_side_to_close,
                            'type': 'MARKET',
                            'quantity': qty_validation['quantity'],
                            'recvWindow': 10000,
                            'newClientOrderId': f"{BOT_INSTANCE_ID}_CLOSE_{position_side_to_close}_{int(time.time())}"
                        }

                        close_response = api_call_with_retry(client.new_order, **close_params)

                        if close_response:
                            print(f"   ✅ Closed {position_side_to_close} position")

                            # Clean up state
                            cleanup_position_state(symbol, position_side_to_close)

                            return True
                        else:
                            print(f"   ❌ Failed to close {position_side_to_close} position")
                            return False

        print(f"   No opposite position found")
        return False

    except Exception as e:
        print(f"❌ Error closing opposite position: {str(e)}")
        traceback.print_exc()
        return False

def cancel_orders_for_position(symbol, position_side=None):
    """
    Cancel all bot's orders for a specific symbol and position side
    Useful for cleanup after position closes
    """
    try:
        print(f"🚫 Canceling orders for {symbol} {position_side or 'ALL'}")

        # Get all open orders for the symbol
        orders_response = api_call_with_retry(client.get_open_orders, symbol=symbol, recvWindow=10000)

        if not orders_response:
            print(f"ℹ️ No open orders found for {symbol}")
            return

        cancelled_count = 0
        bot_orders_found = 0

        for order in orders_response:
            # Check if order belongs to this bot instance
            client_order_id = order.get('clientOrderId', '')
            order_position_side = order.get('positionSide', 'BOTH')

            # Skip orders that don't belong to this bot
            if BOT_INSTANCE_ID not in client_order_id:
                continue

            bot_orders_found += 1

            # If position_side specified, only cancel orders for that position side
            if position_side and position_side != 'BOTH' and order_position_side != position_side:
                continue

            order_id = order['orderId']
            order_type = order.get('type', 'UNKNOWN')

            print(f"🎯 Canceling {order_type} order {order_id} ({order_position_side})")

            cancel_params = {
                'symbol': symbol,
                'orderId': order_id,
                'recvWindow': 6000
            }

            cancel_response = api_call_with_retry(client.cancel_order, **cancel_params)

            if cancel_response:
                cancelled_count += 1
                print(f"✅ Cancelled order {order_id}")
            else:
                print(f"❌ Failed to cancel order {order_id}")

            # Small delay between cancellations
            time.sleep(0.5)

        print(f"📊 Order cleanup summary for {symbol}:")
        print(f"   Bot orders found: {bot_orders_found}")
        print(f"   Orders cancelled: {cancelled_count}")

    except Exception as e:
        print(f"❌ Error canceling orders for {symbol}: {str(e)}")
        import traceback
        traceback.print_exc()

State Management: Logic to keep the bot's internal memory synchronized with the actual exchange positions.

In [ ]:
def track_position(symbol, quantity, entry_price, side, position_side):
    """
    Enhanced position tracking that includes position_side for hedge mode
    Updates the existing track_position function
    """
    position_key = f"{symbol}_{BOT_INSTANCE_ID}_{position_side}"
    tracked_positions[position_key] = {
        'bot_id': BOT_INSTANCE_ID,
        'symbol': symbol,
        'quantity': quantity,
        'entry_price': entry_price,
        'side': side,
        'position_side': position_side,
        'timestamp': datetime.now()
    }
    print(f"📝 Position tracked: {position_key}")

def cleanup_position_state(symbol, position_side=None):
    """
    Clean up bot's internal state after position closes
    Removes tracking data and trailing stop information
    """
    try:
        print(f"🧹 Cleaning up position state for {symbol} {position_side or 'ALL'}")

        # Clean up tracked_positions
        keys_to_remove = []
        for key in tracked_positions.keys():
            if symbol in key:
                if position_side is None or position_side in key:
                    keys_to_remove.append(key)

        for key in keys_to_remove:
            del tracked_positions[key]
            print(f"🗑️ Removed tracked position: {key}")

        # Clean up trailing_stops
        keys_to_remove = []
        for key in trailing_stops.keys():
            if symbol in key:
                if position_side is None or position_side in key:
                    keys_to_remove.append(key)

        for key in keys_to_remove:
            del trailing_stops[key]
            print(f"🗑️ Removed trailing stop: {key}")

        # Clean up symbol_states for this bot
        bot_key = f"{BOT_INSTANCE_ID}_{symbol}"
        if bot_key in symbol_states:
            if position_side:
                # If specific position side, only clear that side's data
                state = symbol_states[bot_key]
                if isinstance(state, dict) and position_side in state:
                    del state[position_side]
                    print(f"🗑️ Removed symbol state for {position_side}")

                    # If no position sides left, remove entire symbol state
                    if not any(side in state for side in ['LONG', 'SHORT']):
                        del symbol_states[bot_key]
                        print(f"🗑️ Removed entire symbol state: {bot_key}")
            else:
                # Remove entire symbol state
                del symbol_states[bot_key]
                print(f"🗑️ Removed symbol state: {bot_key}")

        print(f"✅ Position state cleanup completed for {symbol}")

    except Exception as e:
        print(f"❌ Error in cleanup_position_state: {str(e)}")
        import traceback
        traceback.print_exc()

def monitor_and_cleanup_closed_positions():
    """
    Monitor positions and clean up orders/state when positions are closed
    Should be called periodically in main trading loop
    """
    try:
        # Get current positions from exchange
        positions_response = api_call_with_retry(client.get_position_risk, recvWindow=10000)
        if not positions_response:
            return

        # Create set of active positions (symbol_positionSide)
        active_positions = set()
        for position in positions_response:
            symbol = position['symbol']
            position_side = position.get('positionSide', 'BOTH')
            position_amt = float(position.get('positionAmt', 0))

            if position_amt != 0:
                active_positions.add(f"{symbol}_{position_side}")

        # Check our tracked positions against active positions
        tracked_keys = list(trailing_stops.keys())

        for position_key in tracked_keys:
            # Extract symbol and position side from key
            # Format: "ETHUSDT_ETHVWAPBOT-5_LONG"
            parts = position_key.split('_')
            if len(parts) >= 3:
                symbol = parts[0]
                position_side = parts[-1]  # Last part should be LONG/SHORT

                position_check_key = f"{symbol}_{position_side}"

                # If position is no longer active, clean up
                if position_check_key not in active_positions:
                    print(f"🔔 Position closed detected: {symbol} ({position_side})")

                    # Cancel remaining orders for this position
                    cancel_orders_for_position(symbol, position_side)

                    # Clean up internal state
                    cleanup_position_state(symbol, position_side)

                    # Small delay to prevent overwhelming the API
                    time.sleep(1)

    except Exception as e:
        print(f"❌ Error in monitor_and_cleanup_closed_positions: {str(e)}")
        import traceback
        traceback.print_exc()

Risk Management: Implements daily trade limits to prevent over-trading and protect capital.

In [ ]:
def load_daily_trade_counter():
    """Load daily trade counter from file"""
    try:
        if os.path.exists(TRADE_COUNTER_FILE):
            with open(TRADE_COUNTER_FILE, 'r') as f:
                data = json.load(f)
                return data
        else:
            return {}
    except Exception as e:
        print(f"⚠️ Error loading trade counter: {str(e)}")
        return {}


def increment_daily_trade_count(symbol):
    """Increment today's trade count for a specific symbol"""
    trade_data = load_daily_trade_counter()
    today = get_today_date()
    bot_key = f"{BOT_INSTANCE_ID}_{symbol}"

    if today not in trade_data:
        trade_data[today] = {}

    if bot_key not in trade_data[today]:
        trade_data[today][bot_key] = 0

    trade_data[today][bot_key] += 1

    # Clean up old data (keep only last 7 days)
    cleanup_old_trade_data(trade_data)

    save_daily_trade_counter(trade_data)
    return trade_data[today][bot_key]

def can_trade_today(symbol):
    """Check if we can still trade today (haven't reached daily limit)"""
    current_count = get_daily_trade_count(symbol)
    can_trade = current_count < MAX_TRADES_PER_DAY

    if not can_trade:
        print(f"🚫 {BOT_NAME}: Daily trade limit reached for {symbol} ({current_count}/{MAX_TRADES_PER_DAY})")

    return can_trade

Monitoring & Logging: Functions that provide a visual report of current trades, PnL, and active trailing stops.

In [ ]:
def display_bot_status():
    """
    Display current bot status including positions and orders
    Useful for monitoring and debugging
    """
    try:
        print(f"\n📊 {BOT_NAME} Status Report")
        print("=" * 60)

        # Show tracked positions
        print(f"🎯 Tracked Positions ({len(tracked_positions)}):")
        for key, pos in tracked_positions.items():
            print(f"   {key}: {pos['quantity']} @ {pos['entry_price']:.6f} ({pos['side']})")

        # Show trailing stops
        print(f"\n🔄 Active Trailing Stops ({len(trailing_stops)}):")
        for key, stop in trailing_stops.items():
            symbol = stop['symbol']
            current_sl = stop['current_sl_price']
            print(f"   {key}: SL @ {current_sl:.6f}")

        # Show open orders - get orders for the symbol being traded
        # Since we're trading a single symbol in the main loop, use that
        print(f"\n📋 Open Orders:")

        # Try to get orders for tracked symbols
        all_bot_orders = []

        # Get orders from tracked positions
        tracked_symbols = set()
        for key in tracked_positions.keys():
            # Extract symbol from key (format: "SYMBOL_BOTID_POSITIONSIDE")
            symbol_part = key.split('_')[0]
            tracked_symbols.add(symbol_part)

        # Get orders from trailing stops
        for key in trailing_stops.keys():
            symbol_part = key.split('_')[0]
            tracked_symbols.add(symbol_part)

        # If no tracked symbols, use the main symbol from the trading loop
        if not tracked_symbols:
            # This will be set in the main loop - for now just show message
            print("   No active positions or orders")
        else:
            for sym in tracked_symbols:
                symbol_orders = get_bot_open_orders(sym)
                all_bot_orders.extend(symbol_orders)

            if all_bot_orders:
                for order in all_bot_orders[:5]:  # Show first 5 orders
                    symbol = order['symbol']
                    order_type = order['type']
                    side = order['side']
                    position_side = order.get('positionSide', 'BOTH')
                    print(f"   {symbol} {order_type} {side} ({position_side})")

                if len(all_bot_orders) > 5:
                    print(f"   ... and {len(all_bot_orders) - 5} more orders")
            else:
                print("   No open orders")

        print("=" * 60)

    except Exception as e:
        print(f"❌ Error displaying bot status: {str(e)}")
        import traceback
        traceback.print_exc()

def enhanced_check_position_status(symbols):
    """
    Enhanced position status check with proper trailing stop handling
    """
    try:
        positions = api_call_with_retry(client.get_position_risk, recvWindow=10000)
        if not positions:
            return

        current_time = datetime.now()

        # Check if we're approaching 16:00 ET close time
        et_close_time = current_time.replace(hour=16, minute=0, second=0, microsecond=0)
        if current_time.hour >= 15 and current_time.minute >= 45:  # 15:45 ET warning
            print("🚨 Approaching 16:00 ET close time - preparing to close all positions")

        # Update trailing stops for all active positions
        update_trailing_stop_monitoring()

        for symbol in symbols:
            # Check bot-specific positions
            for position in positions:
                if position['symbol'] == symbol:
                    position_amt = float(position['positionAmt'])
                    if position_amt != 0:
                        # This is an active position
                        position_side = 'LONG' if position_amt > 0 else 'SHORT'
                        bot_position_key = f"{symbol}_{BOT_INSTANCE_ID}_{position_side}"

                        # Check if this position belongs to our bot
                        if bot_position_key in trailing_stops or any(
                                BOT_INSTANCE_ID in key for key in bot_positions if symbol in key):
                            entry_price = float(position['entryPrice'])
                            mark_price = float(position['markPrice'])
                            unrealized_pnl = float(position['unRealizedProfit'])

                            print(f"📊 Monitoring {symbol} {position_side}: {position_amt:.6f} @ {entry_price:.6f}")
                            print(f"   Mark: {mark_price:.6f} | PnL: {unrealized_pnl:.2f}")

                            # Force close at 16:00 ET
                            #if current_time.hour >= 16:
                             #   print(f"🕐 16:00 ET reached - force closing {symbol} position")
                              #  close_position(symbol, position_side)
    except Exception as e:
        print(f"❌ Error in enhanced_check_position_status: {e}")
        import traceback
        traceback.print_exc()



Execution Loop: The main entry point that runs the bot, checks signals, and handles timing between candles.

In [ ]:
symbol = 'BTCUSDT'  # Change to your desired symbol
loop_iteration = 0  # Counter for periodic status display

while True:
    try:
        loop_iteration += 1

        balance, base_volume = get_balance_usdt()
        sleep(1)

        if balance is None or base_volume is None:
            print('Cannot connect to API. Check IP, restrictions or wait some time')
            continue

        print(f"{BOT_NAME}: My balance is: ", balance, " USDT")
        print(f"{BOT_NAME}: Base volume with standard percentage: ", base_volume, " USDT")

        pos = get_bot_positions()
        print(f'{BOT_NAME}: You have {len(pos)} opened positions:\n{pos}')
        enhanced_check_position_status([symbol])

        # 🔄 Update trailing stops for all tracked positions
        update_trailing_stops()

        # 🧹 Monitor and cleanup closed positions (every iteration)
        monitor_and_cleanup_closed_positions()

        ord = check_bot_orders()  # Use bot-specific order checking

        for elem in ord:
            if elem not in pos:
                cancel_bot_orders(elem)  # Use bot-specific cancellation

        # 📊 Display daily trade status
        display_daily_trade_status(symbol)

        # In the main while True loop, replace the signal handling section with:

        if len(pos) < qty_limit:
            signal = combined_strategy_signal(symbol, use_volatility_filter=False, use_bull_bias=False)

            if signal == 'up' and symbol not in ord:
                # Check daily trade limit
                if can_trade_today(symbol):
                    print(f'{BOT_NAME}: Found BUY signal for {symbol}')

                    # Close opposite position if exists
                    close_opposite_position(symbol, 'buy')
                    sleep(2)

                    set_mode(symbol, margin_type)
                    sleep(1)
                    set_leverage(symbol, leverage)
                    sleep(1)
                    print(f'{BOT_NAME}: Placing buy order for {symbol}')

                    order_result = open_order(symbol, 'buy', balance)

                    if order_result:
                        new_count = increment_daily_trade_count(symbol)
                        print(f"✅ {BOT_NAME}: BUY order executed! Daily trades: {new_count}/{MAX_TRADES_PER_DAY}")

                    pos = get_bot_positions()
                    sleep(1)
                    ord = check_bot_orders()
                    sleep(10)
                else:
                    print(f"🚫 {BOT_NAME}: Skipping BUY signal - daily trade limit reached")

            elif signal == 'down' and symbol not in ord:
                # Check daily trade limit
                if can_trade_today(symbol):
                    print(f'{BOT_NAME}: Found SELL signal for {symbol}')

                    # Close opposite position if exists
                    close_opposite_position(symbol, 'sell')
                    sleep(2)

                    set_mode(symbol, margin_type)
                    sleep(1)
                    set_leverage(symbol, leverage)
                    sleep(1)
                    print(f'{BOT_NAME}: Placing sell order for {symbol}')

                    order_result = open_order(symbol, 'sell', balance)

                    if order_result:
                        new_count = increment_daily_trade_count(symbol)
                        print(f"✅ {BOT_NAME}: SELL order executed! Daily trades: {new_count}/{MAX_TRADES_PER_DAY}")

                    pos = get_bot_positions()
                    sleep(1)
                    ord = check_bot_orders()
                    sleep(10)
                else:
                    print(f"🚫 {BOT_NAME}: Skipping SELL signal - daily trade limit reached")

        # 📊 Display bot status every 10 iterations (optional)
        if loop_iteration % 10 == 0:
            display_bot_status()

        sleep_seconds = get_sleep_time()
        current_time = datetime.now()
        next_candle = current_time + timedelta(seconds=sleep_seconds)

        print(f'Current time: {current_time.strftime("%Y-%m-%d %H:%M:%S")}')
        print(f'Waiting until next candle close at: {next_candle.strftime("%Y-%m-%d %H:%M:%S")}')
        print(f'Sleeping for {sleep_seconds} seconds')

        sleep(sleep_seconds)

    except KeyboardInterrupt:
        print(f"🛑 {BOT_NAME} stopped by user")
        # Optional: Cancel all bot orders and clean up on exit
        try:
            cancel_orders_for_position(symbol)
            cleanup_position_state(symbol)
        except:
            pass
        break
    except Exception as e:
        print(f"❌ Error in main trading loop: {str(e)}")
        import traceback

        traceback.print_exc()
        sleep(10)  # Wait before retrying

This is the core decision-making engine. It combines Moving Average crossovers with the Average Directional Index (ADX) to filter out 'fake' signals. It requires a confirmed trend (ADX > 25) and price alignment with the long-term Trend MA before triggering a trade.

In [ ]:
def combined_strategy_signal(symbol, use_volatility_filter=False, use_bull_bias=False):
    """
    Combined MA Crossover + ADX Strategy

    BUY Signal Requirements:
    - Fast MA (9) crosses above Slow MA (18)
    - Price is above Trend MA (100)
    - ADX > 25 (strong trend)

    SELL Signal Requirements:
    - Fast MA (9) crosses below Slow MA (18)
    - Price is below Trend MA (100)
    - ADX > 25 (strong trend)

    Args:
        symbol: Trading symbol
        use_volatility_filter: Enable ATR volatility filter (optional)
        use_bull_bias: Not used in this strategy

    Returns:
        str: 'up' for buy, 'down' for sell, or 0 for no signal
    """
    try:
        print(f"\n{'=' * 60}")
        print(f"📊 Analyzing {symbol} - MA Crossover + ADX Strategy")
        print(f"{'=' * 60}")

        # Get candlestick data
        kl = klines(symbol, candle_interval, limit=150)
        if kl is None or len(kl) < 100:
            print("❌ Insufficient data for analysis")
            return 0

        # Optional: Check volatility filter
        if use_volatility_filter:
            is_acceptable = check_atr_volatility_filter(kl, symbol)
            if not is_acceptable:
                print("🚫 Volatility too high - skipping trade")
                return 0

        # Calculate indicators
        fast_ma, slow_ma, trend_ma = calculate_moving_averages(kl, 9, 18, 200)
        adx = calculate_adx(kl, 14)

        if fast_ma is None or slow_ma is None or trend_ma is None or adx is None:
            print("❌ Failed to calculate indicators")
            return 0

        # Get current values
        current_price = kl['Close'].iloc[-1]
        last_price = kl['Close'].iloc[-2]
        fast_ma_current = fast_ma.iloc[-1]
        slow_ma_current = slow_ma.iloc[-1]
        trend_ma_current = trend_ma.iloc[-1]
        adx_current = adx.iloc[-1]

        # Check for MA crossover
        crossover = check_ma_crossover(fast_ma, slow_ma)

        # Display current market conditions
        print(f"\n📈 Market Conditions:")
        print(f"   Price one candle ago: {last_price:.6f}")
        print(f"   Current Price: {current_price:.6f}")
        print(f"   Fast MA (9): {fast_ma_current:.6f}")
        print(f"   Slow MA (18): {slow_ma_current:.6f}")
        print(f"   Trend MA (100): {trend_ma_current:.6f}")
        print(f"   ADX (14): {adx_current:.2f}")
        print(f"   Crossover: {crossover.upper()}")

        # BUY Signal Logic
        if crossover == 'bullish':
            print(f"\n🔍 Bullish crossover detected!")

            # Check if price is above trend MA
            price_above_trend = current_price > trend_ma_current
            print(f"   Price above 100 MA: {'✅' if price_above_trend else '❌'}")

            # Check if ADX is strong
            adx_strong = adx_current > 25
            print(f"   ADX > 25: {'✅' if adx_strong else '❌'} ({adx_current:.2f})")

            if price_above_trend and adx_strong:
                print(f"\n✅ BUY SIGNAL CONFIRMED for {symbol}")
                print(f"   All conditions met:")
                print(f"   - Fast MA crossed above Slow MA ✅")
                print(f"   - Price above Trend MA ✅")
                print(f"   - Strong trend (ADX > 25) ✅")
                return 'up'
            else:
                print(f"\n⚠️ Bullish crossover but conditions not met")
                return 0

        # SELL Signal Logic
        elif crossover == 'bearish':
            print(f"\n🔍 Bearish crossover detected!")

            # Check if price is below trend MA
            price_below_trend = current_price < trend_ma_current
            print(f"   Price below 100 MA: {'✅' if price_below_trend else '❌'}")

            # Check if ADX is strong
            adx_strong = adx_current > 25
            print(f"   ADX > 25: {'✅' if adx_strong else '❌'} ({adx_current:.2f})")

            if price_below_trend and adx_strong:
                print(f"\n✅ SELL SIGNAL CONFIRMED for {symbol}")
                print(f"   All conditions met:")
                print(f"   - Fast MA crossed below Slow MA ✅")
                print(f"   - Price below Trend MA ✅")
                print(f"   - Strong trend (ADX > 25) ✅")
                return 'down'
            else:
                print(f"\n⚠️ Bearish crossover but conditions not met")
                return 0

        else:
            print(f"\n⏸️ No crossover detected - waiting for signal")
            return 0

    except Exception as e:
        print(f"❌ Error in combined_strategy_signal: {str(e)}")
        import traceback
        traceback.print_exc()
        return 0

def calculate_moving_averages(kl, fast_period=9, slow_period=18, trend_period=200):
    """
    Calculate fast, slow, and trend moving averages

    Args:
        kl: DataFrame with OHLC data
        fast_period: Fast MA period (default 9)
        slow_period: Slow MA period (default 18)
        trend_period: Trend MA period (default 100)

    Returns:
        tuple: (fast_ma, slow_ma, trend_ma)
    """
    try:
        fast_ma = kl['Close'].rolling(window=fast_period).mean()
        slow_ma = kl['Close'].rolling(window=slow_period).mean()
        trend_ma = kl['Close'].rolling(window=trend_period).mean()

        return fast_ma, slow_ma, trend_ma
    except Exception as e:
        print(f"Error calculating moving averages: {e}")
        return None, None, None

def calculate_adx(kl, period=14):
    """
    Calculate ADX (Average Directional Index)

    Args:
        kl: DataFrame with High, Low, Close columns
        period: ADX calculation period (default 14)

    Returns:
        Series: ADX values
    """
    try:
        adx_indicator = ta.trend.ADXIndicator(
            high=kl['High'],
            low=kl['Low'],
            close=kl['Close'],
            window=period
        )
        adx = adx_indicator.adx()
        return adx
    except Exception as e:
        print(f"Error calculating ADX: {e}")
        return None


def check_ma_crossover(fast_ma, slow_ma, lookback=2):
    """
    Check for moving average crossover

    Args:
        fast_ma: Fast moving average series
        slow_ma: Slow moving average series
        lookback: Number of candles to check for crossover (default 2)

    Returns:
        str: 'bullish', 'bearish', or 'none'
    """
    try:
        # Get recent values
        fast_current = fast_ma.iloc[-1]
        fast_previous = fast_ma.iloc[-2]
        slow_current = slow_ma.iloc[-1]
        slow_previous = slow_ma.iloc[-2]

        # Bullish crossover: fast crosses above slow
        if fast_previous <= slow_previous and fast_current > slow_current:
            return 'bullish'

        # Bearish crossover: fast crosses below slow
        if fast_previous >= slow_previous and fast_current < slow_current:
            return 'bearish'

        return 'none'
    except Exception as e:
        print(f"Error checking MA crossover: {e}")
        return 'none'